In [5]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import requests
import json
from scipy.stats import linregress
from config import weather_api_key_sab

In [2]:
# Import the clean ufo data

ufo_df = pd.read_csv("Resources/usa_sights.csv")

# Keep only the columns needed for this analysis
ufo_df = ufo_df[["city", "state", "year", "sightings", "date_time"]]

# Format ufo dataframe for merging with income dataframe
#ufo_df["state"] = ufo_df["state"].str.upper()
#ufo_df = ufo_df.rename(columns={"state": "State", "sightings": "Sightings", "year": "Year"})

# Convert year column to string for concatenation
ufo_df["year"] = ufo_df["year"].astype(str)

# Empty list to store date
ymd = []
time = []

for index, row in ufo_df.iterrows():
    # Drop cities with spaces to facilitiate API calls
    test = row["city"].split(" ")
    if len(test)>1:
        ufo_df = ufo_df.drop(index=index)
        
    else:
        # Get date in proper format to call API
        day = row["date_time"].split(" ")
        ymd.append(row["year"]+"-"+day[0])

        # Get time in proper format for API
        hours = day[1].split(":")
        #time.append(hours[0]+":00:00")
        time.append(hours[0])

ufo_df["date"] = ymd
ufo_df["time"] = time

# Convert time column to int to index API response
ufo_df["time"] = ufo_df["time"].astype(int)

ufo_df["state"] = ufo_df["state"].str.upper()
ufo_df["city"] = ufo_df["city"].str.title()

# Only look at dates from 1970 and later (the API dates back to 1970)
ufo_df = ufo_df.loc[ufo_df["year"]>="1970"]

ufo_df

,city,state,year,sightings,date_time,date,time
9,Bellmore,NY,1970,1,10-10 16:00:00,1970-10-10,16
10,Manchester,KY,1970,1,10-10 19:00:00,1970-10-10,19
11,Lexington,NC,1971,1,10-10 21:00:00,1971-10-10,21
14,Niantic,CT,1973,1,10-10 19:00:00,1973-10-10,19
15,Hudson,MA,1974,1,10-10 19:30:00,1974-10-10,19
...,...,...,...,...,...,...,...
63548,Nashville,TN,2013,1,09-09 21:15:00,2013-09-09,21
63549,Boise,ID,2013,1,09-09 22:00:00,2013-09-09,22
63550,Napa,CA,2013,1,09-09 22:00:00,2013-09-09,22
63551,Vienna,VA,2013,1,09-09 22:20:00,2013-09-09,22


In [3]:
# Get a population sample
ufo_sample = ufo_df.sample(n=45)
ufo_sample

,city,state,year,sightings,date_time,date,time
44734,Tigard,OR,2012,1,07-14 22:00:00,2012-07-14,22
32653,Knoxville,TN,2004,1,04-08 23:40:00,2004-04-08,23
13074,Tucson,AZ,2006,1,11-08 04:50:00,2006-11-08,4
43885,Fontana,CA,1990,1,07-01 23:00:00,1990-07-01,23
23344,Allendale,CA,2008,1,02-28 22:00:00,2008-02-28,22
2283,Waterford,MI,2006,1,10-18 23:00:00,2006-10-18,23
43833,Caldwell,WI,1980,1,07-01 01:00:00,1980-07-01,1
7004,Waterville,ME,2012,1,11-01 19:20:00,2012-11-01,19
32298,Richmond,VA,2007,1,04-05 21:00:00,2007-04-05,21
17458,Austin,TX,2009,1,12-31 23:46:00,2009-12-31,23


In [7]:
# Create variables for storing weather information
temp = []
humidity = []
cloudcover = []
visibility = []
moonphase = []

# Track the number of iterations
count = 0

# Interate through all ufo sightings and get weather information
for index, row in ufo_sample.iterrows():
    count += 1
    # Try to call API for given city and time
    try:
        city = row["city"] + "," + row["state"]
        date = row["date"]
        url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}/{date}?unitGroup=us&key={weather_api_key_sab}"
        response = requests.get(url).json()
        # Store necessary information
        temp.append(response["days"][0]["hours"][row["time"]]["temp"])
        humidity.append(response["days"][0]["hours"][row["time"]]["humidity"])
        cloudcover.append(response["days"][0]["hours"][row["time"]]["cloudcover"])
        visibility.append(response["days"][0]["hours"][row["time"]]["visibility"])
        moonphase.append(response["days"][0]["moonphase"])
        print(count, "getting data for", city)
    # If not able to call, store NaN to later drop that row
    except:
        temp.append("NaN")
        humidity.append("NaN")
        cloudcover.append("NaN")
        visibility.append("NaN")
        moonphase.append("NaN")
        print(count, "no info for", city)

1 no info for Tigard,OR
2 no info for Knoxville,TN
3 no info for Tucson,AZ
4 no info for Fontana,CA
5 no info for Allendale,CA
6 no info for Waterford,MI
7 no info for Caldwell,WI
8 no info for Waterville,ME
9 no info for Richmond,VA
10 no info for Austin,TX
11 no info for Plano,TX
12 no info for Glendale,CA
13 no info for Wrentham,MA
14 no info for Columbus,IN
15 no info for Tavares,FL
16 no info for Waynesboro,PA
17 no info for Miami,FL
18 no info for Connersville,IN
19 no info for Brentwood,CA
20 no info for Golden,CO
21 no info for Dexter,MO
22 no info for Tonasket,WA
23 no info for Richmond,VA
24 no info for Seattle,WA
25 no info for Branford,CT
26 no info for Scottsdale,AZ
27 no info for Ames,IA
28 no info for Portland,OR
29 no info for Phoenix,AZ
30 no info for Frontenac,MN
31 no info for Hollywood,CA
32 no info for Navarre,FL
33 no info for Piqua,OH
34 no info for Greensburg,PA
35 no info for Parrish,FL
36 no info for Galveston,TX
37 no info for Vista,CA
38 no info for Seattle,

In [ ]:
# Update ufo data frame to include weather infomation
ufo_sample["temp"] = temp
ufo_sample["humidity"] = humidity
ufo_sample["cloudcover"] = cloudcover
ufo_sample["visibility"] = visibility
ufo_sample["moonphase"] = moonphase

ufo_sample.head()

In [ ]:
city = ufo_df["city"][160] + "," + ufo_df["state"][160]
date = ufo_df["date"][160]
print(city)
print(date)

In [ ]:
# Call Weather API to get current city and date weather info
url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{date}/{date}?unitGroup=us&key={weather_api_key_fran}"
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=False))

In [ ]:
# Store necessary information
temp = response["days"][0]["hours"][ufo_df["time"][160]]["temp"]
humidity = response["days"][0]["hours"][ufo_df["time"][160]]["humidity"]
cloudcover = response["days"][0]["hours"][ufo_df["time"][160]]["cloudcover"]
visibility = response["days"][0]["hours"][ufo_df["time"][160]]["visibility"]
moonphase = response["days"][0]["moonphase"]